# Stacking of models into two layers
1. First layer: train models on the whole training set
2. Second layer: train a model on the first layer's predictions and the rest of the training set

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import KFold

from src.features.preprocess_data import get_preprocessed_test_data, fetch_preprocessed_data
pd.set_option('display.max_columns', 200)
from sklearn.ensemble import RandomForestRegressor


from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings('ignore')


%pip install xgboost

## Get the data

In [ ]:
X_train_obs_combined, X_val_obs_combined, y_train_obs_combined, y_val_obs_combined, X_train_est_combined, X_val_est_combined, y_train_est_combined, y_val_est_combined = fetch_preprocessed_data()
x_test_whole = get_preprocessed_test_data()

x_whole = pd.concat([X_train_obs_combined, X_val_obs_combined])
y_whole = pd.concat([y_train_obs_combined, y_val_obs_combined])
x_whole.reset_index(drop=True, inplace=True)
y_whole.reset_index(drop=True, inplace=True)

x_whole.head()

## Train test split for the base layer and meta layer

In [ ]:
# The base layer gets 80% of the data
# The meta layer gets 20% of the data
base_to_meta_layer_split = 0.8

base_x_train = x_whole.sample(frac=base_to_meta_layer_split)
meta_x_train = x_whole.sample(frac=1-base_to_meta_layer_split)

# Get the corresponding y values
base_y_train = y_whole[base_x_train.index]
meta_y_train = y_whole[meta_x_train.index]
base_x_train.columns

# Train Base level models
It is important to use a variety of models to get a diverse set of predictions.

I want a model to check if there is a linear relationship between the location features and the target. I will use a linear regression model for this.
I want to check if the different irradiation values are correlated with the target. I will use xgboost for this.
I want to check if the different temperature values are correlated with the target. I will use 


## XGBoost

In [ ]:
# K-fold cross validation
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

total_mae = 0

xgboost_models = []


for train_index, test_index in kf.split(base_x_train):

    reg = xgb.XGBRegressor(n_estimators=100000,
                       early_stopping_rounds=50,
                       learning_rate= 0.01,
                       objective="reg:linear",
                       eval_metric="mae",
                       sub_sample = 0.9,
                       colsample_bytree = 0.8,
                       gamma = 0,
                       alpha = 0.001,
                       min_child_weight=0,
                       max_depth=9)

    X_train, X_test = base_x_train.iloc[train_index], base_x_train.iloc[test_index]
    y_train, y_test = base_y_train.iloc[train_index], base_y_train.iloc[test_index]

    reg.fit(X_train, y_train,
            eval_set=[(X_val_est_combined, y_val_est_combined)],
            verbose=100)
    
    xgboost_models.append(reg)
    predictions = reg.predict(X_test)
    
    mae = mean_absolute_error(y_test, predictions)
    total_mae += mae
    
    print(f"Fold {total_mae}, Mean Absolute Error: {mae}")

average_mse = total_mae / num_folds
print(f"Average Mean Squared Error: {average_mse}")


## Catboost

In [ ]:
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import KFold
import numpy as np

num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

total_mae = 0
catboost_models = []

def compute_sample_weight(data):
    # Assign weight of 2 for estimated data and 1 for observed data
    return np.where(data['time_since_prediction'] > 0, 2, 1)

for train_index, test_index in kf.split(base_x_train):
    reg = CatBoostRegressor(
        iterations=10000000,
        depth=8,
        learning_rate=0.001,
        loss_function='MAE',
        verbose=200
    )
    
    X_train, X_test = base_x_train.iloc[train_index], base_x_train.iloc[test_index]
    y_train, y_test = base_y_train.iloc[train_index], base_y_train.iloc[test_index]
    
    # Compute sample weights for training and testing data
    train_weight = compute_sample_weight(X_train)
    test_weight = compute_sample_weight(X_test)

    # Create Pool for training and testing
    train_pool = Pool(data=X_train, label=y_train, weight=train_weight)
    test_pool = Pool(data=X_test, label=y_test, weight=test_weight)

    # Fit the model using the sample weights
    reg.fit(train_pool, eval_set=test_pool, early_stopping_rounds=100)

    catboost_models.append(reg)
    predictions = reg.predict(test_pool)
    
    # Compute weighted MAE manually
    weighted_mae = np.sum(test_weight * np.abs(y_test - predictions)) / np.sum(test_weight)
    total_mae += weighted_mae
    
    print(f"Fold {len(catboost_models)}, Weighted Mean Absolute Error: {weighted_mae}")

average_mae = total_mae / num_folds
print(f"Average Weighted Mean Absolute Error: {average_mae}")


## lightgbm

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import KFold
import pandas as pd

# Assuming x_whole and y_whole are defined and are pandas DataFrames

num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
total_mae = 0

lightgbm_models = []

params = {
    'objective': 'regression_l1',
    'boosting_type': 'gbdt',
    'max_depth': 8,
    'metric': 'mae',
    'num_leaves': 256,
    'learning_rate': 0.001,
    'feature_fraction': 1.0,
    'bagging_fraction': 0.9,
    'bagging_freq': 5,
    'min_data_in_leaf': 20,
    'early_stopping_rounds': 100,
    'verbosity': 100,  # 0 for verbose, -1 for silent
}

num_round = 100000 # number of training iterations

# Ensure column names are compatible with LightGBM
base_x_train.columns = base_x_train.columns.str.replace('[^A-Za-z0-9_]', '_', regex=True)

for train_index, test_index in kf.split(base_x_train):
    X_train, X_test = base_x_train.iloc[train_index], base_x_train.iloc[test_index]
    y_train, y_test = base_y_train.iloc[train_index], base_y_train.iloc[test_index]

    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_test, label=y_test)

    reg = lgb.train(params, train_data, num_round, valid_sets=[valid_data])
    lightgbm_models.append(reg)
    predictions = reg.predict(X_test)
    
    mae = mean_absolute_error(y_test, predictions)
    total_mae += mae
    
    print(f"Fold {total_mae}, Mean Absolute Error: {mae}")

average_mse = total_mae / num_folds
print(f"Average Mean Squared Error: {average_mse}")

## Random Forest

In [ ]:
def average_prediction(x_values :pd.DataFrame, models) -> pd.DataFrame:
    """
    Function for predicting on multiple models and averaging the results
    """
    results = models[0].predict(x_values)
    for model in models[1:]:
        model: xgb.XGBRegressor
        prediction = model.predict(x_values)
        results += prediction
    
    results = results / len(models)

    return results


# Train meta learner model

Create dataset for meta learner model by using models to predict on the meta layer training set

In [ ]:
# Predict on the base layer on meta_x_train
base_xgboost_predictions = average_prediction(meta_x_train, xgboost_models)
base_catboost_predictions = average_prediction(meta_x_train, catboost_models)
base_lightgbm_predictions = average_prediction(meta_x_train, lightgbm_models)
# base_random_forest_predictions = average_prediction(meta_x_train, reg_models)

# Add the predictions to the meta_x_train
meta_x_train["xgboost"] = base_xgboost_predictions
meta_x_train["catboost"] = base_catboost_predictions
meta_x_train["lightgbm"] = base_lightgbm_predictions
# meta_x_train["random_forest"] = base_random_forest_predictions



Train meta learner model on new dataset

In [ ]:
# K-fold cross validation

num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

total_mae = 0

meta_models = []


for train_index, test_index in kf.split(meta_x_train):

    reg = xgb.XGBRegressor(n_estimators=100000,
                       early_stopping_rounds=50,
                       learning_rate= 0.01,
                       objective="reg:linear",
                       eval_metric="mae",
                       sub_sample = 0.9,
                       colsample_bytree = 0.8,
                       gamma = 0,
                       alpha = 0.001,
                       min_child_weight=0,
                       max_depth=9)

    X_train, X_test = meta_x_train.iloc[train_index], meta_x_train.iloc[test_index]
    y_train, y_test = y_whole.iloc[train_index], y_whole.iloc[test_index]

    reg.fit(X_train, y_train,
            eval_set=[(X_val_est_combined, y_val_est_combined)],
            verbose=100)
    
    meta_models.append(reg)
    predictions = reg.predict(X_test)
    
    mae = mean_absolute_error(y_test, predictions)
    total_mae += mae
    
    print(f"Fold {total_mae}, Mean Absolute Error: {mae}")

average_mse = total_mae / num_folds
print(f"Average Mean Squared Error: {average_mse}")

In [ ]:
fi = pd.DataFrame(data=reg.feature_importances_,
             index=reg.feature_names_in_,
             columns=["importance"])

plt.figure(figsize=(100,100))
plt.tight_layout()
fi.sort_values("importance").plot(kind="barh", title="Feature Importance")
plt.show()

In [ ]:
y_pred_val_obs_combined = average_prediction(X_val_obs_combined, meta_models)
y_pred_val_est_combined = average_prediction(X_val_est_combined, meta_models)

# Evaluate the model's performance using Mean Absolute Error (MAE) on the combined validation observed data
mae_obs_combined = mean_absolute_error(y_val_obs_combined, y_pred_val_obs_combined)
mae_est_combined = mean_absolute_error(y_val_est_combined, y_pred_val_est_combined)
print('MAE on validation observed data: ', mae_obs_combined)
print('MAE on validation estimated data: ', mae_est_combined)


## Get predictions for meta learner model test set

In [ ]:
# Predict on the cleaned validation set
from src.features.preprocess_data import get_final_prediction

y_predictions = average_prediction(x_test_whole, meta_models)

## Save predictions

In [ ]:
from src.models.saving import save_predictions


save_predictions(y_predictions, 'stacking with possible data leakage')